In [ ]:
import pandas as pd

### Load the data

In [ ]:
data = pd.read_csv("../data/alzheimers_disease_data_balanced_gpt.csv")
data = data.drop(
    "DoctorInCharge", axis=1
)  # this attribute is confidential in the data, and thus not useful
eth_col = data.pop("Ethnicity")
data.insert(3, "Ethnicity", eth_col)

data.head()

In [ ]:
data.isna().sum() # check for missing values

## CGAN Model

In [ ]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata

metadata = Metadata.detect_from_dataframe(
    data = data, table_name="patient_data")

synthesizer = CTGANSynthesizer(metadata)
synthesizer.fit(data, epochs = 400, verbose=True)

In [11]:
import torch
torch.cuda.device_count()

0